In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import os
import warnings
import seaborn as sns

import matplotlib.colors as mcolors
import ast

warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("../0_dataset/dataset.csv")
df.head()

In [ ]:
# calculate the correlation matrix
corr_matrix = X.corr().abs()
corr_matrix

# fueature selection
# Drop features with correlation greater than 0.9
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.9)]
to_drop = set(to_drop)
to_drop = [item for item in to_drop if item in df.columns]
X = X.drop(columns=to_drop)

X.head()

In [ ]:
from sklearn.feature_selection import RFE, RFECV
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

from skopt import BayesSearchCV
from sklearn.model_selection import cross_val_score, cross_val_predict, LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [ ]:
# 

# Linear Regression 
linear_params = {
    'fit_intercept': [True, False],
    'copy_X': [True, False]
}

# Ridge 
ridge_params = {
    'alpha': (0.005, 10.0),
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'max_iter': (100, 500),
    'tol': (1e-5, 1e-0),
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],

}

# Decision Tree
# decision_tree_params = {
#     'criterion': ['absolute_error', 'squared_error', 'friedman_mse'],
#     'splitter': ['best', 'random'],
#     'max_depth': (1, 30),
#     'min_samples_split': (2, 20),
#     'min_samples_leaf': (1, 20),
#     'max_features': ['sqrt', 'log2'],
#     'min_impurity_decrease': (0, 0.5),
#     'random_state': (0, 100)
# }

# # Extra Tree 
# extra_tree_params = {
#     'criterion': ['absolute_error', 'squared_error', 'friedman_mse'],
#     'splitter': ['best', 'random'],
#     'max_depth': (1, 30),
#     'min_samples_split': (2, 10),
#     'min_samples_leaf': (1, 10),
#     'min_weight_fraction_leaf': (0, 0.5),
#     'max_features': ['sqrt', 'log2'],
#     'min_impurity_decrease': (0, 0.5),
#     #'warm_start': [True, False],
#     'random_state': (0, 100),
#     'ccp_alpha': (0, 1.0)
# }

# Random Forest 
random_forest_params = {
    'n_estimators': (10, 500),
    'criterion': ['absolute_error', 'squared_error', 'friedman_mse'],
    'max_depth': (1, 100),
    'min_samples_split': (2, 100),
    'min_samples_leaf': (1, 100),
    'max_features': ['sqrt', 'log2'],
    'random_state': (0, 100)
}

# Ada Boost 
ada_boost_params = {
    'n_estimators': (10, 1000), 
    'learning_rate': (0.005, 1.0),
    'random_state': (0, 100),
    'loss': ['linear', 'square', 'exponential']
}

# Gradient Boosting 
gradient_boosting_params = {
    'n_estimators': (100, 500),
    'learning_rate': (0.005, 1.0),
    'loss': ['huber', 'squared_error', 'quantile', 'absolute_error'],
    'subsample': (0.5, 1.0),
    'max_depth': (1, 100),
    'min_samples_split': (2, 100),
    'min_samples_leaf': (1, 100),
    'max_features': ['sqrt', 'log2'],
    'n_iter_no_change': (20, 50),
    'random_state': (0, 100),
    'criterion': ['squared_error', 'friedman_mse']
}

# LGBM 
lgbm_params = {
    'n_estimators': (100, 500),
    'learning_rate': (0.01, 1.0),
    'num_leaves': (10, 100),
    'max_depth': (1, 100),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'min_child_samples': (1, 10),
    'min_child_weight': (0.001, 0.1),
    'reg_alpha': (0.0, 1.0),
    'reg_lambda': (0.0, 1.0),
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 200),
    'max_depth': (2, 10),
    'num_leaves': (10, 30),
    'min_data_in_leaf': (10, 30),
    'lambda_l1': (0.1, 10.0),
    'lambda_l2': (0.1, 10.0),
    'bagging_fraction': (0.5, 1.0),
    'bagging_freq': (1, 2, 3),
    'feature_fraction': (0.5, 1.0),
    'random_state': (0, 100)
}

# XGB 
xgb_params = {
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 1000),
    'max_depth': (3, 50),
    'min_child_weight': (1, 3),
    'gamma': (0.1, 10.0),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'lambda': (0.1, 10.0),
    'alpha': (0.1, 10.0),
    'random_state': (0, 100)
}




In [ ]:
regressors = [
    ('Gradient Boosting Regressor', GradientBoostingRegressor(), gradient_boosting_params),
    ('Random Forest Regressor', RandomForestRegressor(), random_forest_params),
    ('Ada Boost Regressor', AdaBoostRegressor(), ada_boost_params),
    ('Linear Regression', LinearRegression(), linear_params),
    # ('K Neighbors Regressor', KNeighborsRegressor()),
    # ('SVR', SVR(C=1000,gamma=0.01, kernel='rbf')),#C=1000000
    ('Ridge', Ridge(),ridge_params),
    #('Lasso', Lasso(alpha=0.1, max_iter=100,random_state=42)),
    # ('MLP Regressor', MLPRegressor(hidden_layer_sizes=(160,80), activation='relu', solver='lbfgs', learning_rate_init=0.001, max_iter=1000, random_state=42)),
    # ('Decision Tree Regressor', DecisionTreeRegressor(random_state=42), decision_tree_params),
    # ('Extra Tree Regressor', ExtraTreeRegressor(random_state=42), extra_tree_params),
    #('Bagging Regressor', BaggingRegressor(estimator=RandomForestRegressor(), random_state=42))
    ('LGBM Regressor', LGBMRegressor(), lgbm_params),
    ('XGB Regressor', XGBRegressor(), xgb_params)
]

In [ ]:
model_params = {}

results = {}
feature_results = {}
cv_results = {}


for name, regressor, params in regressors:

    if name in ['Decision Tree Regressor', 'Extra Tree Regressor']:
        step = 0.5
        min_features_to_select = 5
    elif name in ["Ridge"]:
        step = 1
        min_features_to_select = 5
    elif name in ["Ada Boost Regressor"]:
        step = 0.3
        min_features_to_select = 4
    elif name in ["Gradient Boosting Regressor"]:
        step = 0.30
        min_features_to_select = 4
    elif name in [ "LGBM Regressor"]:
        step = 0.3
        min_features_to_select = 5
    elif name in ['XGB Regressor']:
        step = 0.3
        min_features_to_select = 5
    elif name in ['Random Forest Regressor']:
        step = 0.3
        min_features_to_select = 4
    else: 
        step = 0.2
        min_features_to_select = 5


    selector = RFECV(regressor, step=step, cv=10, min_features_to_select=min_features_to_select)
    X_wrapper = selector.fit_transform(X, y)
    

    # 
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=)
    train_x_w, test_x_w, train_y, test_y = train_test_split(X_wrapper, y, test_size=0.1, random_state=)
    # 
    scaler = StandardScaler()
    train_x = scaler.fit_transform(train_x)
    test_x = scaler.transform(test_x)
    train_x_w = scaler.fit_transform(train_x_w)
    test_x_w = scaler.transform(test_x_w)
    
    print("Regressor is %s" % name)
    print("N_features %s" % selector.n_features_)
    print("Support is %s" % selector.support_)
    print("Ranking %s" % selector.ranking_)


    feature_results[name] = {
        'n_features': selector.n_features_,
        'support': selector.support_,
        'ranking': selector.ranking_
    }
    

    for i in feature_results:
        idx_list = []
        for idx, item in enumerate(feature_results[i]["support"]):
            if item == True:
                idx_list.append(idx)

    feature_names = []
    for i in idx_list:
        feature_names.append(X.columns[i])
    print("Feature names is %s" % feature_names)




    search_space = params

    cv = 10

    search = BayesSearchCV(estimator=regressor, search_spaces=search_space, cv=cv, n_jobs=-1)





    search.fit(train_x_w, train_y)
    model_w = regressor.set_params(**search.best_params_)
    print(model_w.get_params())

    model_params[name] =  model_w.get_params()


    model_w.fit(train_x_w, train_y)
    pred_y_train_w = model_w.predict(train_x_w)
    pred_y_test_w = model_w.predict(test_x_w)

    train_r2_w = r2_score(train_y, pred_y_train_w)
    test_r2_w = r2_score(test_y, pred_y_test_w)
    rmse_train_w = np.sqrt(mean_absolute_error(train_y, pred_y_train_w))
    rmse_test_w = np.sqrt(mean_absolute_error(test_y, pred_y_test_w))


    cv_scores_w = cross_val_score(model_w, train_x_w, train_y, cv=10, scoring='neg_mean_absolute_error')
    cv_rmse_w = np.sqrt(-cross_val_score(model_w, train_x_w, train_y, cv=10, scoring='neg_mean_squared_error'))
    r2_scores_w = cross_val_score(model_w, train_x_w, train_y, cv=10, scoring='r2')
    avg_mae_w = -np.mean(cv_scores_w)
    avg_rmse_w = np.mean(cv_rmse_w)
    avg_r2_w = np.mean(r2_scores_w)

    loo_predicted_values = cross_val_predict(model_w, train_x_w, train_y.values, cv=LeaveOneOut())
    loo_mae = cross_val_score(model_w, train_x_w, train_y.values, cv=LeaveOneOut(), scoring='neg_mean_squared_error')
    loo_rmse = np.sqrt(-loo_mae)
    loo_r2 = cross_val_score(model_w, train_x_w, train_y.values, cv=LeaveOneOut(), scoring='r2')
    loo_predicted_series = pd.Series(loo_predicted_values)




    if name in ["Ridge", "Linear Regression"]:
        feature_importance = model_w.coef_
    else:
        feature_importance = model_w.feature_importances_
    

    results[name] = {
        'train_r2_w': float(f"{train_r2_w:.4f}"), 'test_r2_w': float(f"{test_r2_w:.4f}"), 
        'rmse_train_w': float(f"{rmse_train_w:.4f}"), 'rmse_test_w': float(f"{rmse_test_w:.4f}"),
        'mae_train_w': float(f"{mean_absolute_error(train_y, pred_y_train_w):.4f}"), 'mae_test_w': float(f"{mean_absolute_error(test_y, pred_y_test_w):.4f}"),
        'cv_avg_mae_w': float(f"{avg_mae_w:.4f}"), 'cv_avg_rmse_w': float(f"{avg_rmse_w:.4f}"), 'cv_avg_r2_w': float(f"{avg_r2_w:.4f}"),
        'loo_mae': float(f"{mean_absolute_error(train_y, loo_predicted_values):.4f}"), 'loo_r2': float(f"{r2_score(train_y, loo_predicted_values):.4f}"),
        "n_features": int(selector.n_features_),
        'model_params': model_w.get_params(),
        'feature_importance': feature_importance.tolist(),
        'feature_names': feature_names
    }


    print('-------------------------------------------------------------')
    print(name)
    print("\nSelected input:")
    print(f"Train r2_w: {train_r2_w:.4f}, Test r2_w: {test_r2_w:.4f}")
    print(f"Train RMSE_w: {rmse_train_w:.4f}, Test RMSE_w: {rmse_test_w:.4f}")
    print("\nCross validation:")
    print(f"CV avg MAE_w: {avg_mae_w:.4f}, CV avg RMSE_w: {avg_rmse_w:.4f}, CV avg R2_w: {avg_r2_w:.4f}")
    print('MAE_LOOCV:', mean_absolute_error(train_y, loo_predicted_values))
    print('R2_LOOCV:', r2_score(train_y, loo_predicted_values))
    print('-------------------------------------------------------------')
    print('loo_mae:', loo_mae)
    print('loo_rmse:', loo_rmse)
    print('loo_r2:', loo_r2)
    print("cv_mae:", cv_scores_w)
    print('cv_rmse_w:', cv_rmse_w)
    print('r2_scores_w:', r2_scores_w)
    print('-------------------------------------------------------------')
        

    fig, ax = plt.subplots(figsize=(10,10))
    ax.set_xticklabels(feature_names, rotation=45, ha='right')
    ax.bar(x=range(len(feature_importance)), height=feature_importance, tick_label=feature_names, color="#1d91c0")

    ax.set_ylabel('Feature importance')
    ax.bar(x=range(len(feature_importance)), height=feature_importance, tick_label=feature_names)
    plt.savefig(f'{name}_feature_importance.svg', dpi=600, format='svg')
    plt.show()


    

    fig, ax = plt.subplots(figsize=(10,10))

    plt.rc('font',family='Calibri', size=22)
    
    train = ax.scatter(train_y, pred_y_train_w, s=100, c='blue')
    test = ax.scatter(test_y, pred_y_test_w, s=100, c='red')
    #val = ax.scatter(val_y, pred_y_val, s=100, c='pink')
    ax.plot([-1.5, 0.5],[-1.5, 0.5], c='black', linewidth=3.5)
    ax.plot([-1.5, 0.4], [-1.4, 0.5], c='black', linewidth=2, linestyle='--')
    ax.plot([-1.4, 0.5], [-1.5, 0.4], c='black', linewidth=2, linestyle='--')

    ax.legend((train, test), ('Training set', 'Test set'), loc='lower right')
    ax.set_xlim(-1.5, 0.5)
    ax.set_ylim(-1.5, 0.5)
    ax.set_xlabel('ΔΔG / (kcal/mol)')
    ax.set_ylabel('Predicted ΔΔG / (kcal/mol)')
    ax.text(x=-1.45, y=-0.2,
        # s=f"{name}\nTrain r2: {train_r2:.4f}\nTest r2: {test_r2:.4f}\nTrain RMSE: {rmse_train:.4f}\nTest RMSE: {rmse_test:.4f}\nMAE_LOOCV: {mean_absolute_error(train_y, loo_predicted_values):.4f}\nR2_LOOCV: {r2_score(train_y, loo_predicted_values):.4f}")
        # s = "{name}\nTrain r²: {train_r2:.4f}\nTest r²: {test_r2:.4f}\nTrain RMSE: {rmse_train:.4f}\nTest RMSE: {rmse_test:.4f}\nMAE_{LOOCV}: {mean_absolute_error(train_y, loo_predicted_values):.4f}\nR²$_{LOOCV}$: {r2_score(train_y, loo_predicted_values):.4f}"
        s = '\n{}\nTrain $R^2$ = {:.4f} \n Test $R^2$ = {:.4f}\n$R^2_L$ = {:.4f}\nTrain MAE = {:.4f}\nTest MAE = {:.4f}\nLOO MAE = {:.4f}'.format(name, train_r2_w, test_r2_w, r2_score(train_y, loo_predicted_values), mean_absolute_error(train_y, pred_y_train_w), mean_absolute_error(test_y, pred_y_test_w),mean_absolute_error(train_y, loo_predicted_values)),
        fontsize=22
    )
    plt.savefig(f'{name}.svg', dpi=600, format='svg')
    plt.show()




print(results)



